In [1]:
import requests
import os
import json
from pathlib import Path
from typing import Dict
from skema.program_analysis.JSON2GroMEt.json2gromet import json_to_gromet

## Generate endpoint URL

In [2]:
SKEMA_ADDRESS = os.environ.get("SKEMA_ADDRESS", "http://skema-py:8000")
URL = f"{SKEMA_ADDRESS}/code2fn/fn-given-filepaths"

## Process a single file 

In [3]:
post_data = {
    "files":["exp0.py"],
    "blobs":["x=2"]
}

response_single = requests.post(URL, json=post_data)
print(response_single.json())

{'schema': 'FN', 'schema_version': '0.1.7', 'name': '', 'modules': [{'schema': 'FN', 'schema_version': '0.1.7', 'name': 'exp0', 'fn': {'name': None, 'b': [{'function_type': 'MODULE', 'body': None, 'import_type': None, 'import_version': None, 'import_source': None, 'source_language': None, 'source_language_version': None, 'value': None, 'gromet_type': 'GrometBoxFunction', 'name': 'module', 'metadata': 3}], 'opi': None, 'opo': None, 'wopio': None, 'bf': [{'function_type': 'EXPRESSION', 'body': 1, 'import_type': None, 'import_version': None, 'import_source': None, 'source_language': None, 'source_language_version': None, 'value': None, 'gromet_type': 'GrometBoxFunction', 'name': None, 'metadata': 5}], 'pif': None, 'pof': [{'id': 1, 'name': 'x', 'box': 1, 'default_value': None, 'gromet_type': 'GrometPort', 'metadata': None}], 'wfopi': None, 'wfl': None, 'wff': None, 'wfc': None, 'wfopo': None, 'bl': None, 'pil': None, 'pol': None, 'wlopi': None, 'wll': None, 'wlf': None, 'wlc': None, 'wlop

## Process a system

In [4]:
def system_to_json(
    root: Path, system_filepaths: Path, system_name: str
) -> Dict:
    supported_extensions = [".py", ".F", ".f95"]
    files = []
    blobs = []
    for file in system_filepaths.read_text().split("\n"):
        path_obj = Path(root, file)
        if path_obj.suffix in supported_extensions:
            files.append(file)
            blobs.append(path_obj.read_text())

    return {
            "files": files,
            "blobs": blobs,
            "system_name": system_name,
            "root_name": root.stem
    }


post_data = system_to_json(Path("/data", "skema", "code", "system_test"), Path("/data", "skema", "code", "system_test", "system_filepaths.txt"), "system_test")
response_multi = requests.post(URL, json=post_data)
print(response_multi.json())

{'schema': 'FN', 'schema_version': '0.1.7', 'name': 'system_test', 'modules': [{'schema': 'FN', 'schema_version': '0.1.7', 'name': 'driver', 'fn': {'name': None, 'b': [{'function_type': 'MODULE', 'body': None, 'import_type': None, 'import_version': None, 'import_source': None, 'source_language': None, 'source_language_version': None, 'value': None, 'gromet_type': 'GrometBoxFunction', 'name': 'module', 'metadata': 3}], 'opi': None, 'opo': None, 'wopio': None, 'bf': [{'function_type': 'EXPRESSION', 'body': None, 'import_type': None, 'import_version': None, 'import_source': None, 'source_language': None, 'source_language_version': None, 'value': None, 'gromet_type': 'GrometBoxFunction', 'name': 'model', 'metadata': None}], 'pif': None, 'pof': [{'id': 1, 'name': 'model', 'box': 1, 'default_value': None, 'gromet_type': 'GrometPort', 'metadata': None}], 'wfopi': None, 'wfl': None, 'wff': None, 'wfc': None, 'wfopo': None, 'bl': None, 'pil': None, 'pol': None, 'wlopi': None, 'wll': None, 'wlf'

## Write output JSON to file

In [5]:
outfile = Path("/data", "skema", "code", "exp0--Gromet-FN-auto.json")
with open(outfile, "w") as f:
    f.write(json.dumps(response_single.json()))

## Import Gromet JSON

In [6]:
gromet_module_collection = json_to_gromet(outfile)